# Function Calling Demo: Enabling a LLM to talk to a router

## Overview
This notebook demonstrates how the ``OpenAI`` function call works for the GPT models and how it can be used to programmatically interact with a Cisco device.

For programmatic access to the router, this notebook uses ``pyATS``. This library unifies interaction with Cisco devices by using code to execute commands and parse output. This demo uses an IOS-XE router in an always-on Cisco Devnet sandbox. Please make sure that you can ssh to ``sandbox-iosxe-latest-1.cisco.com``. Location dependent (e.g., when on VPN) access might be blocked.

_Created by: Lukas Marschhausen_

## Getting Started

To start, we import all the necessary libraries that we need.

The required libraries are listed in the requirements.txt file.

⚠ _pyATS is not yet available for Windows. Try using the WSL (Windows System for Linux) or a Linux VM to ensure a smooth pyATS experience_

In [1]:
import os
import json

# pyATS: To access a Cisco device Programatically.
from pyats.topology import loader
from genie.conf import Genie

# OpenAI: To access ChatGPT's API
from openai import OpenAI

## Using the OpenAI API

Now let us make a test call to the OpenAI API (ChatGPT)

In [2]:
# Obtain our API key from the .env file
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Create an OpenAI Client Object
client = OpenAI(api_key=(openai_api_key))

# Send a request to the API
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "You are part of a demonstration on stage at Cisco Live, how do you feel? Answer like a funny network engineer. Answer briefly.",
        }
    ],
    model="gpt-3.5-turbo"
)

# Extracting the return Message
print(chat_completion.choices[0].message.content.replace('. ', '.\n'))

I feel like a packet lost in transit, but hey, at least I'm getting some stage time!


## Using pyATS

#### Configuring pyATS to execute commands on a router

We need a ``testbed.yaml`` file where we define our devices and how to connect to them.

```
testbed:
  name: Testbed
  credentials:
    default:
      username: admin
      password: C1sco12345

devices:
  Cat8000V:
    alias: Cat8000V
    os: iosxe
    type: iosxe
    connections:
      defaults:
        class: unicon.Unicon
      cli:
        ip: sandbox-iosxe-latest-1.cisco.com
        port: 22
        protocol: ssh
```

#### Connecting to the router
⚠ _Make sure that you can ssh to sandbox-iosxe-latest-1.cisco.com. Access from corporate networks etc. might be restricted (e.g., check whether you are on VPN if you're having trouble to connect)._

In [3]:
# First we load the Testbed
testbed = loader.load('testbed.yaml')
genie_testbed = Genie.init(testbed)

# Select the Device we want to interact with
device = genie_testbed.devices["Cat8000V"]

# Connect to the device
device.connect()


2024-01-29 05:28:54,453: %UNICON-INFO: +++ Cat8000V logfile /tmp/Cat8000V-cli-20240129T052854452.log +++

2024-01-29 05:28:54,454: %UNICON-INFO: +++ Unicon plugin iosxe (unicon.plugins.iosxe) +++

This is me practicing with ansible
Hello friends
How are we doing today?


2024-01-29 05:28:55,235: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.143 -p 22, id: 140711067719184 +++

2024-01-29 05:28:55,238: %UNICON-INFO: connection to Cat8000V
(admin@131.226.217.143) Password: 

This is me practicing with ansible1
Hello friends
How are we doing today?



Cat8000V#

2024-01-29 05:28:56,142: %UNICON-INFO: +++ initializing handle +++

2024-01-29 05:28:56,219: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term length 0' +++
term length 0
Cat8000V#

2024-01-29 05:28:56,774: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term width 0' +++
term width 0
Cat8000V#

2024-01-29 05:28:57,162: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 's

'\r\r\nThis is me practicing with ansible\r\r\nHello friends\r\r\nHow are we doing today?\r\r\n\n\r(admin@131.226.217.143) Password: \r\n\r\nThis is me practicing with ansible1\r\nHello friends\r\nHow are we doing today?\r\n\r\n\r\n\r\nCat8000V#\nterm length 0\r\nCat8000V#\nterm width 0\r\nCat8000V#\nshow version\r\nCisco IOS XE Software, Version 17.09.02a\r\nCisco IOS Software [Cupertino], Virtual XE Software (X86_64_LINUX_IOSD-UNIVERSALK9-M), Version 17.9.2a, RELEASE SOFTWARE (fc4)\r\nTechnical Support: http://www.cisco.com/techsupport\r\nCopyright (c) 1986-2022 by Cisco Systems, Inc.\r\nCompiled Wed 30-Nov-22 02:47 by mcpre\r\n\r\n\r\nCisco IOS-XE software, Copyright (c) 2005-2022 by cisco Systems, Inc.\r\nAll rights reserved.  Certain components of Cisco IOS-XE software are\r\nlicensed under the GNU General Public License ("GPL") Version 2.0.  The\r\nsoftware code licensed under GPL Version 2.0 is free software that comes\r\nwith ABSOLUTELY NO WARRANTY.  You can redistribute and/or

#### Executing Commands on the Device

In [4]:
# Now we can run commands on the device
output = device.execute("show version")
print (output)


2024-01-29 05:28:59,123: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'show version' +++
show version
Cisco IOS XE Software, Version 17.09.02a
Cisco IOS Software [Cupertino], Virtual XE Software (X86_64_LINUX_IOSD-UNIVERSALK9-M), Version 17.9.2a, RELEASE SOFTWARE (fc4)
Technical Support: http://www.cisco.com/techsupport
Copyright (c) 1986-2022 by Cisco Systems, Inc.
Compiled Wed 30-Nov-22 02:47 by mcpre


Cisco IOS-XE software, Copyright (c) 2005-2022 by cisco Systems, Inc.
All rights reserved.  Certain components of Cisco IOS-XE software are
licensed under the GNU General Public License ("GPL") Version 2.0.  The
software code licensed under GPL Version 2.0 is free software that comes
with ABSOLUTELY NO WARRANTY.  You can redistribute and/or modify such
GPL code under the terms of GPL Version 2.0.  For more details, see the
documentation or "License Notice" file accompanying the IOS-XE software,
or the applicable URL provided on the flyer accompanying the IOS-XE
softwa

#### Disconnecting from the Device

In [5]:
device.disconnect()

## OpenAI Function calling

#### Now lets create a run_command function for ChatGPT to call...

```
run_command(device, command): execute command on specified device
```

In [6]:
def run_command(device, command):
    try:
        # Load testbed
        testbed = loader.load('testbed.yaml')
        genie_testbed = Genie.init(testbed)

        # Get device information and connect
        device = genie_testbed.devices[device]
        device.connect()

        # Run command
        output = device.execute(command)

        # Disconnect and return
        device.disconnect()
        return output
    except Exception as e:
        print("Error running Command: " + str(e))
        return

_Note: You would not normally close the connection after each command if you plan to run multiple commands. This is for demonstration purposes only._

In [7]:
print(run_command("Cat8000V", "show interfaces"))


2024-01-29 05:29:10,845: %UNICON-INFO: +++ Cat8000V logfile /tmp/Cat8000V-cli-20240129T052910843.log +++

2024-01-29 05:29:10,849: %UNICON-INFO: +++ Unicon plugin iosxe (unicon.plugins.iosxe) +++

This is me practicing with ansible
Hello friends
How are we doing today?


2024-01-29 05:29:11,704: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.143 -p 22, id: 140711067470928 +++

2024-01-29 05:29:11,710: %UNICON-INFO: connection to Cat8000V
(admin@131.226.217.143) Password: 

This is me practicing with ansible1
Hello friends
How are we doing today?



Cat8000V#

2024-01-29 05:29:12,657: %UNICON-INFO: +++ initializing handle +++

2024-01-29 05:29:12,735: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term length 0' +++
term length 0
Cat8000V#

2024-01-29 05:29:13,291: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term width 0' +++
term width 0
Cat8000V#

2024-01-29 05:29:13,690: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 's

#### Giving ChatGPT the information about ``run_command()``.

To do this, we first need to define the ``tools`` that ChatGPT can use

In [8]:
tools =[
        {
            "type": "function",
            "function": {
                "name": "run_command",
                "description": "Executes a command on a device and gets the output.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "device": {
                            "type": "string",
                            "description": "The name of the device. This is case sensitive. If no device is specified, use Cat8000V.",
                        },
                        "command": {
                            "type": "string",
                            "description": "The command to run on the Device. The Device uses the iosxe operating system. Always run the commands that get the most information.",
                        },
                    },
                    "required": ["device", "command"],
                },
            },
        }
    ]

#### Now we can call ChatGPT and tell it to use ``tools``.

```tool_choice="auto"```: Lets ChatGPT decide whether to call a tool or not. 

``tool_choice='{"type": "function", "function": {"name": "run_command"}}'``: force the GPT to **always** use the tool 

In [9]:
messages=[
    {
        "role": "user",
        "content": "You are a network expert. Tell me what interfaces are on this device?"
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice=
    {
    "type": "function",
    "function": { "name": "run_command" },
    }
)

first_assistant_response = response.choices[0].message
tool_call = response.choices[0].message.tool_calls[0]

# Get the Name of the Function
function_name = tool_call.function.name

# Get the Arguments of the Function and parse them
function_arguments = tool_call.function.arguments
function_arguments = json.loads(function_arguments)


print("Function: " + function_name)
print("Arguments: " + str(function_arguments))


Function: run_command
Arguments: {'device': 'Cat8000V', 'command': 'show ip interface brief'}


```ChatGPT wants to call run_command```
#### So we call the function...

In [10]:
function_response = ""

if function_name == "run_command":

    # Obtain the individual Arguments of that were requested
    device = function_arguments.get("device")
    command = function_arguments.get("command")

    # Run the function with the Arguments ChatGPT provided
    function_response = run_command(device, command)

print (function_response)


2024-01-29 05:29:28,173: %UNICON-INFO: +++ Cat8000V logfile /tmp/Cat8000V-cli-20240129T052928170.log +++

2024-01-29 05:29:28,175: %UNICON-INFO: +++ Unicon plugin iosxe (unicon.plugins.iosxe) +++

This is me practicing with ansible
Hello friends
How are we doing today?


2024-01-29 05:29:28,951: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.143 -p 22, id: 140711038981008 +++

2024-01-29 05:29:28,954: %UNICON-INFO: connection to Cat8000V
(admin@131.226.217.143) Password: 

This is me practicing with ansible1
Hello friends
How are we doing today?



Cat8000V#

2024-01-29 05:29:29,865: %UNICON-INFO: +++ initializing handle +++

2024-01-29 05:29:29,942: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term length 0' +++
term length 0
Cat8000V#

2024-01-29 05:29:30,490: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term width 0' +++
term width 0
Cat8000V#

2024-01-29 05:29:30,881: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 's

#### Returning the function output to ChatGPT
``Important``: We need to provide the **full conversation history** up to this point for the OpenAI function call to work.

Each tool that is called is assigned an ID. We need to reference this ID in the response we send.

In [11]:
# Add ChatGPT's first Resonse to the Messages.
messages.append(first_assistant_response)

# Add the Result of out function call to the Messages.
messages.append(
    {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": function_name,
        "content": function_response,
    }
)

# Now we can send that to ChatGPT and let it continue the response with the information provided by the function. 
second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )

print(second_response.choices[0].message.content)

The device has the following interfaces:
- GigabitEthernet1 with IP address 10.10.20.48
- GigabitEthernet2 with IP address 10.10.10.12
- GigabitEthernet3 (administratively down)
- Loopback0 with IP address 10.0.0.1
- Loopback5 with IP address 172.31.1.1
- Loopback10
- Loopback21
- Loopback50 with IP address 1.1.254.254
- Loopback109 with IP address 10.255.255.9
- VirtualPortGroup0 with IP address 192.168.1.1


## Now let's put it all together...

This is an example of how it would be implemented in a real scenario.


Here ChatGPT can also call a function two or more times to gather more information.

```run_command("Cat8000V", "show interfaces")```

```run_command("Cat8000V", "show version")```

In [12]:
def run_conversation():
    messages = [{"role": "user", "content": "You are a network professional. Give me a quick list of all the important information about the device."}]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "run_command": run_command,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function name: {function_name}, arguments: {function_args}")
            function_response = function_to_call(
                device=function_args.get("device"),
                command=function_args.get("command"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        print(f"----------------------------------------\n\n")
        return second_response
    
print(run_conversation().choices[0].message.content)

Function name: run_command, arguments: {'device': 'Cat8000V', 'command': 'show version'}

2024-01-29 05:29:49,543: %UNICON-INFO: +++ Cat8000V logfile /tmp/Cat8000V-cli-20240129T052949542.log +++

2024-01-29 05:29:49,544: %UNICON-INFO: +++ Unicon plugin iosxe (unicon.plugins.iosxe) +++

This is me practicing with ansible
Hello friends
How are we doing today?


2024-01-29 05:29:50,323: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.143 -p 22, id: 140711038792464 +++

2024-01-29 05:29:50,325: %UNICON-INFO: connection to Cat8000V
(admin@131.226.217.143) Password: 

This is me practicing with ansible1
Hello friends
How are we doing today?



Cat8000V#

2024-01-29 05:29:51,223: %UNICON-INFO: +++ initializing handle +++

2024-01-29 05:29:51,298: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term length 0' +++
term length 0
Cat8000V#

2024-01-29 05:29:51,844: %UNICON-INFO: +++ Cat8000V with via 'cli': executing command 'term width 0' +++
term width 0
Cat8000V#
